In [ ]:
"""
Homework 1

Aaron Santucci
Professor VanderLinden
CS-344
"""


In [ ]:
"""
1.  The earliest psychologists studied the human mind using introspection. 
    Would this be a good way to inform efforts in AI to model human cognitive processes? 
    (Please give a one-to-two-paragraph answer.) 
"""

"""
    I don't think that introspection is a good model for AI. Introspection is an abstract concept that relies
    on the ability to acknowledge and reflect on one's own mental state. Given that the current models for AI either seem
    to be related to search algorithms or machine learning, I don't think that the current level of technology could keep
    up with the many facets of a biological brain that make us able to do introspection right now.
"""


In [ ]:
""" 
2.  Implement a local search formulation of a TSP problem and use the 
    AIMA hill-climbing and simulated annealing implementations to solve it.
"""

In [ ]:

"""
Travelling Sales Person problem using hill climbing and simulated annealing
https://en.wikipedia.org/wiki/Travelling_salesman_problem

@author: ajs94
@version 20feb2019
"""

import time
from tools.aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule
from random import randrange


class TSP(Problem):

    def __init__(self, mapSize, pathsList):
        self.mapSize = mapSize
        self.pathsList = pathsList

        # Starting point is random
        self.initial = []
        self.initial.append(randrange(0, mapSize - 1))
        print("Starting point:\t" + str(self.initial[0]) + "\n")

    def actions(self, state):
        actions = []
        for point in range(0, mapSize):
            if point not in state:
                actions.append(point)
        return actions

    def result(self, state, move):
        new_state = state[:]
        new_state.append(move)
        return new_state

    def value(self, state):
        value = 0
        for i in range(0, len(state) - 1):
            spot = (i + 1) % mapSize
            value += self.pathsList[tuple(sorted((state[i], state[spot])))]
        return value


if __name__ == '__main__':

    # for size anything below about 20-30 the time will be ~0
    mapSize = 100
    pathsList = {}

    # assign each path links and randomized distances with other points
    # ae '(0, 1): 10' is point 0's link to point 1 with a distance of 10
    for i in range(0, mapSize):
        for k in range(i + 1, mapSize):
            path = (i, k)
            path_dist = randrange(1, 11)
            pathsList[path] = path_dist

    # don't over-clutter the screen with paths and distances
    if mapSize <= 10:
        print('Paths:\t' + str(pathsList ))

    problem = TSP(mapSize, pathsList)

    t = time.time()
    hill_climbing = hill_climbing(problem)
    hc_time = time.time() - t

    print('Hill climbing:\t' + str(hill_climbing)
          + '\n\tvalue: ' + str(problem.value(hill_climbing))
          + '\n\ttime: ' + str(hc_time)
          )

    t = time.time()
    sim_annealing = simulated_annealing(problem, exp_schedule(k=20, lam=0.005, limit=1000))
    sa_time = time.time() - t

    print('Simulated annealing\t: ' + str(sim_annealing )
          + '\n\tvalue: ' + str(problem.value(sim_annealing ))
          + '\n\ttime: ' + str(sa_time)
          )


In [ ]:

"""
3.Formulate a course-scheduling domain and implement it using the AIMA constraint satisfaction framework.
"""


In [ ]:

"""
a course-scheduling domain and implement it using the AIMA constraint satisfaction framework
https://cs.calvin.edu/courses/cs/344/kvlinden/03constraint/homework.html

@author: ajs94
@version 21feb2019
"""

import time
from csp import min_conflicts, \
    CSP, parse_neighbors


class Scheduler:

    def __init__(self, courses, schedule, staff, rooms):
        self.schedule = schedule
        self.staff = staff
        self.rooms = rooms

        self.possible_setups = []
        self.variables = courses
        self.domains = {}
        self.neighbors = {}

    # variables: a list of variables; each is atomic (e.g. int or string).
    def get_variables(self):
        return self.variables

    # domains: a dict of {var:[possible_value, ...]} entries.
    def get_domains(self):
        for prof in self.staff:
            for timeslot in self.schedule:
                for room in self.rooms:
                    self.possible_setups.append(prof + " " + timeslot + " " + room)
        for var in self.variables:
            self.domains[var] = self.possible_setups

        return self.domains

    # neighbors: a dict of {var:[var,...]} that for each variable lists
    #   the other variables that participate in constraints.
    def get_neighbors(self):
        self.neighbors = parse_neighbors("junk: filler")
        for type in [self.variables, self.possible_setups]:
            for A in type:
                for B in type:
                    if A != B:
                        if B not in self.neighbors[A]:
                            self.neighbors[A].append(B)
                        if A not in self.neighbors[B]:
                            self.neighbors[B].append(A)

        return self.neighbors

    # constraints: a function f(A, a, B, b) that returns true if neighbors
    #   A, B satisfy the constraint when they have values A=a, B=b
    def constraint_check(self, A, a, B, b):
        # split[0] = professor, split[1] = time, split[2] = room
        splitA = a.split()
        splitB = b.split()
        # Check if same professor and time
        if ((splitA[0] == splitB[0]) and (splitA[1] == splitB[1])):
            return False
        # Check if same time and room
        if ((splitA[1] == splitB[1]) and (splitA[2] == splitB[2])):
            return False

        return True

    def get_csp(self):
        return CSP(self.get_variables(), self.get_domains(), self.get_neighbors(), self.constraint_check)


if __name__ == '__main__':

    # the stuff to be scheduled ...
    courses = ["CS-108", "CS-112", "CS-212", "CS-214", "CS-232", "CS-336", "CS-344"]
    timeslots = ["MWF9AM", "MWF1030AM", "MWF130PM", "TTH1030AM", "TTH130PM"]
    professors = ["Plantinga", "Adams", "Norman", "VanderLinden", "Schuurman"]
    classrooms = ["SB-410", "SB-411"]

    t = time.time()
    Scheduler = Scheduler(courses, timeslots, professors, classrooms)
    solution = min_conflicts(Scheduler.get_csp())
    total_time = time.time() - t

    if solution is not None:
            print('Schedule found:\t' + str(solution)
          + '\n\ttime: ' + str(total_time)
          )
